In [1]:
from sklearn import datasets, linear_model, neighbors, svm, ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from base import SuperLearner
from base import BMA
import pandas as pd
import numpy as np
from pyearth import Earth
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) # warnings from py-earth

v_folds = 5
ols = linear_model.LinearRegression()
elnet = linear_model.ElasticNetCV(l1_ratio=0.5, cv=v_folds, normalize=True)
ridge = linear_model.RidgeCV(cv=v_folds)
lars = linear_model.LarsCV(cv=v_folds, normalize=True)
lasso = linear_model.LassoCV(cv=v_folds, normalize=True)
nn = neighbors.KNeighborsRegressor(weights='distance')
svm1 = svm.SVR(kernel='linear', C=10, gamma='auto')
svm2 = svm.SVR(kernel='poly', C=10, gamma='auto')
rf = ensemble.RandomForestRegressor(n_estimators=100,min_samples_split=5)
gbm = ensemble.GradientBoostingRegressor()
# earth is sort of like D/S/A?
earth=Earth(max_terms=50,max_degree=3,use_fast=True,verbose=0) # get this from https://github.com/scikit-learn-contrib/py-earth
rtree=DecisionTreeRegressor()

def sim1(n):
    w=np.random.binomial(1,.4,size=(10,n))
    eps=np.random.normal(0,1,size=n)
    y=2*w[0]*w[9]+4*w[1]*w[6]+3*w[3]*w[4]-\
    5*w[5]*w[9]+3*w[7]*w[8]+w[0]*w[1]*w[3]-\
    2*w[6]*(1-w[5])*w[1]*w[8]-4*(1-w[9])*w[0]*(1-w[3])+eps
    return np.transpose(w),y

train1,test1=sim1(500),sim1(10000)

c:\program files\python36\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
c:\program files\python36\lib\site-packages\numpy\.libs\libopenblas.BNVRK7633HSX7YVO2TADGR4A5KEKXJAW.gfortran-win_amd64.dll
c:\program files\python36\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# instantiate the super learner!
SuperLearner(cand_learners=[ols,lars,earth,rf],V=10
            ).debug(*train1,*test1)

,Learner,Train MSE,Train CV MSE,Test MSE,Coefs
0,LinearRegression,5.492732,5.754549,6.528144,1.440416
1,LarsCV,5.492732,5.748391,6.528143,-1.458934
2,Earth,0.991621,1.079697,1.155161,1.028644
3,RandomForestRegressor,0.706997,1.918009,1.930600,-0.041181
4,Meta (LinearRegression),1.024071,1.075465,1.161744,NaN


In [3]:
SuperLearner(cand_learners=[ols,lars,earth,rf],V=10,meta_learner=lasso
            ).debug(*train1,*test1)

,Learner,Train MSE,Train CV MSE,Test MSE,Coefs
0,LinearRegression,5.492732,5.754549,6.528144,0.00000
1,LarsCV,5.492732,5.748391,6.528143,0.00000
2,Earth,0.991621,1.079697,1.155161,0.98595
3,RandomForestRegressor,0.728524,1.861291,1.972125,0.00000
4,Meta (LassoCV),0.993431,1.079554,1.156213,NaN


In [4]:
SuperLearner(cand_learners=[ols,lars,earth,rf],V=10,meta_learner=rtree
            ).debug(*train1,*test1)

,Learner,Train MSE,Train CV MSE,Test MSE
0,LinearRegression,5.492732,5.754549,6.528144
1,LarsCV,5.492732,5.748391,6.528143
2,Earth,0.991621,1.079697,1.155161
3,RandomForestRegressor,0.723640,1.876283,1.922879
4,Meta (DecisionTreeRegressor),1.874953,0.015444,2.296512


In [5]:
diabetes=datasets.load_diabetes()

SuperLearner(cand_learners=[ols,lars,earth,rf],V=10,meta_learner=lasso
            ).debug(diabetes.data,diabetes.target)

,Learner,Train MSE,Train CV MSE,Coefs
0,LinearRegression,2859.690399,2999.032286,0.660394
1,LarsCV,2932.128955,3849.042608,0.083387
2,Earth,2576.797876,3214.796738,0.230406
3,RandomForestRegressor,629.140920,3441.925542,0.000000
4,Meta (LassoCV),2735.124829,2962.192314,NaN
